In [4]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import pandas as pd
import re  # 숫자추출을 위한 라이브러리
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep


In [5]:
def create_excel(time, rank, server) : 
# 엑셀 만들기
    wb = openpyxl.Workbook()

    # 워크시트 만들기
    ws = wb.active
    ws.title = f"최근 {time}일 {server}서버의 {rank}랭크"

    # 데이터 추가하기
    ws.append(['직업', '덱유형', '게임수'])

    # 엑셀 저장하기
    wb.save('./hs_meta.xlsx')

    # 엑셀 열기
    wb = openpyxl.load_workbook('./hs_meta.xlsx')
    ws = wb.active


In [10]:
def driver_in(): 
# ChromeOptions 설정
  chrome_options = Options()
  chrome_options.add_experimental_option("detach", True)
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')

  # ChromeDriverManager를 사용하여 Chrome 드라이버를 초기화합니다.
  driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

  # url 열기
  url = "https://hsreplay.net/account/login"
  driver.get(url)
  sleep(3)
  
  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  try:
      element_present = EC.presence_of_element_located((By.CLASS_NAME, "promo-button"))
      WebDriverWait(driver, 10).until(element_present)
  except Exception as e:
      print(f"엘리먼트 대기 중 예외 발생: {e}")
      driver.quit()

  btn = driver.find_element(By.CLASS_NAME,"promo-button")
  print("btn",btn)
  btn.click()

  # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
  try:
      element_present = EC.presence_of_element_located((By.CLASS_NAME, "input-block"))
      WebDriverWait(driver, 10).until(element_present)
  except Exception as e:
      print(f"엘리먼트 대기 중 예외 발생: {e}")
      driver.quit()

  id = driver.find_elements(By.CLASS_NAME, "input-block")[0]
  id.send_keys("")
  pw = driver.find_elements(By.CLASS_NAME, "input-block")[1]
  pw.send_keys("")
  log_btn = driver.find_element(By.CLASS_NAME, "submit-button")
  log_btn.click()

In [31]:
def scrape_hs_meta_data():
    # 입력값 받기
    time = int(input("검색할 시간을 입력하세요.\n[최근7일 : 7 // 최근 3일 : 3 // 최근 1일 : 1 // 최신 패치 : 0]"))
    rank = input("검색할 랭크를 입력하세요.\n[전구간 : A // 브론즈 : B // 실버 : S // 골드 : G // 플래티넘 : P // 다이아 : D // 전설 : L // 상위1000 : 1000]")
    server = input("검색할 서버를 입력하세요.\n[전서버 : A // 아시아 : AP // 유럽 : EU // 북미 : US]")
    
    # 검색 범위 딕셔너리 정의
    time_dict={7 : "LAST_7_DAY", 3 :  "LAST_3_DAY", 1 : "LAST_1_DAY", 0 : ""}
    rank_dict={"A" : "", "B" :  "BRONZE", "S" : "SILVER", "G" :  "GOLD", "P" :  "PLATINUM", "D" :  "DIAMOND", "L" :  "LEGEND", "1000" :  "TOP_1000_LEGEND"}
    server_dict={"A" : "", "AP" :  "REGION_KR", "EU" : "REGION_EU", "US" : "REGION_US"}

    create_excel(time, rank_dict[rank], server_dict[server])

    # ChromeOptions 설정
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')

    # ChromeDriverManager를 사용하여 Chrome 드라이버를 초기화합니다.
    driver = webdriver.Chrome(service=ChromeService(executable_path=ChromeDriverManager().install()), options=chrome_options)

    # URL 열기
    if rank_dict[rank]=="":
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time_dict[time]}&region={server_dict[server]}"
    else:
      url = f"https://hsreplay.net/meta/#tab=archetypes&timeFrame={time_dict[time]}&rankRange={rank_dict[rank]}&region={server_dict[server]}"
    driver.get(url)

    # 엘리먼트가 나타날 때까지 대기합니다 (최대 10초 대기)
    try:
        element_present = EC.presence_of_element_located((By.CLASS_NAME, 'box-title'))
        WebDriverWait(driver, 10).until(element_present)
    except Exception as e:
        print(f"엘리먼트 대기 중 예외 발생: {e}")
        driver.quit()

    # BeautifulSoup을 사용하여 HTML 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    #  엘리먼트 선택
    hero_elements = soup.select('.box-title')
    type_elements = soup.select('.tooltip-wrapper')
    game_elements = soup.select('.table-cell')

    print('.box-title', hero_elements)
    print('.tooltip-wrapper', type_elements)   
    print('.table-cell', game_elements)      

In [11]:
driver_in()

btn <selenium.webdriver.remote.webelement.WebElement (session="7d622f7a83469aad5f6558119ecbca9f", element="DA04782692E37C770705ABC3266E545B_element_39")>
